# ANÁLISE DE ATIVIDADES DE MANUTENÇÃO E DE CONFIABILIDADE DE ATIVOS DE UMA EMPRESA FABRICANTE DE SINALIZAÇÃO PARA VEÍCULOS PESADOS E COMERCIAIS

### Ana Luiza Figueiredo Gouvea

# Visão Geral (2019 - 2021)

Este trabalho visa analisar os dados do setor de manutenção de uma empresa fabricante de sinalização para veículos pesados e comerciais, gerando informações sobre a confiabilidade dos ativos e também sobre os resultados da atuação do setor de manutenção ao decorrer dos anos.  Para isso, serão analisadas informações referentes aos principais tipos de atividades realizadas pelo setor, sendo elas classificadas como corretivas, preventivas e melhorias.

Para a análise dos dados entre janeiro de 2019 e maio de 2021, foram usadas planilhas geradas por um software terceiro contratado pela empresa. Os dados foram gerados em 6 planilhas no formato xlsx. Atualmente, a empresa não possui mais acesso ao software e essas planilhas encontram-se salvas no banco de dados interno do setor de manutenção. 
O software terceiro gera planilhas sem separação por tipo de atividade. 


---

### Importação das bibliotecas 

In [ ]:
# Instalações de pacotes:
!pip install pyspark 
!pip install jupyter 
!pip install findspark 

#Serão necessários na parte de analise de analise de palavras -> Utilizado na parte de analise das atividades 'corretivas'.

In [ ]:
# -*- coding: utf-8 -*-
# Bibliotecas para analise dos dados:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime as dt

# -----------------------------------------
#Instalação das bibliotecas para analise de palavras -> Utilizado na parte de analise das atividades 'corretivas':

import findspark
import pyspark
import nltk
import string
import collections
from collections import Counter
#---------------------------------------------
# Impressão das Versões das Bibliotecas utilizadas:
print(f'Matplot version: {matplotlib.__version__}')
print(f'Numpy version: {np.__version__}')
print(f'Pandas version: {pd.__version__}')
print(f'Seaborn version: {sns.__version__}')
print(f'Findspark version: {findspark.__version__}')
print(f'Pyspark version: {pyspark.__version__}')
print(f'NLTK version: {nltk.__version__}')

#--------------------------------------------

print('Download das bibliotecas completo!')

### Importação dos arquivos

In [ ]:
# Caminho para a pasta em que os arquivos estão:
path = r'CAMINHO PARA A PASTA EM QUE OS ARQUIVOS XLSX ANALISADOS ESTÃO'
files  = os.listdir(path) #listando os arquivos encontrados
df = pd.DataFrame()

In [ ]:
# Arquivos encontrados dentro da pasta:
print(files)

In [ ]:
#Para garantir que iremos analizar apenas arquivos xlsx:
files_xlsx = (path + '\\' + f for f in files if f[-4:] == 'xlsx')

In [ ]:
# Concatenação em um unico DataFrame para análise:
for f in files_xlsx:
    data = pd.read_excel(f) # lendo os arquivos xlsx
    df = df.append(data) # adicionando todos em um mesmo dataframe 'df'

In [ ]:
#Informações do DataFrame:
df.info()

In [ ]:
# Visualização das primeiras linhas do DataFrame:
df.head()

### Tratamento de Dados

Vamos filtrar apenas as colunas que iremos utilizar em nossas análise, sendo elas: 'Ativo', 'Tipo de ativo' ,classificados como instalação (Infraestrutura) ou Equipamento (máquina), a 'Tarefa', o 'Tipo de Tarefa', e 'Data Final' da atividade.

In [ ]:
# Criação do arquivo com as colunas que serão utilizadas:
arquivo_final = df[['Ativo','Tipo de Ativo','Tarefa','Tipo de tarefa','Data final']]
arquivo_final.head()

In [ ]:
# Informações do arquivo:
arquivo_final.info()

In [ ]:
#Foi criado uma coluna com valor 1 para auxiliar nas contagens:
arquivo_final.insert(loc=0,column='contador',value=1)
arquivo_final

# ** Análise do Período **

In [ ]:
# Filtrar dados no período que se deseja:
periodo = '2019'  # AQUI É COLOCADO O ANO QUE SE DESEJA ANALISAR
arquivo_final_periodo = arquivo_final.loc[arquivo_final['Data final'].str.contains(periodo,na = False)]
arquivo_final_periodo

### Tipos de tarefas mais executadas

In [ ]:
# Tipos de Tarefa:
tipos_tarefas_mais_realizadas = arquivo_final_periodo[['Tipo de tarefa','contador']].groupby('Tipo de tarefa').sum()
tipos_tarefas_mais_realizadas = tipos_tarefas_mais_realizadas.sort_values(by=['contador'],ascending=False)
tipos_tarefas_mais_realizadas

In [ ]:
#Grafico Tipos de Tarefas mais executadas no período:

tipos_tarefas_mais_realizadas = tipos_tarefas_mais_realizadas.head(10)
indice_x = tipos_tarefas_mais_realizadas['contador']
nomes_tarefas = (tipos_tarefas_mais_realizadas.index)
plt.figure(figsize=(15,10))
sns.set_style('white') # deixar o fundo do grafico branco. Caso o código seja aplicado em um compilador com a tela de fundo escura.
figura_tarefas = sns.barplot(x=indice_x,y=nomes_tarefas,palette = 'inferno')
figura_tarefas.set_title('Os 10 tipos de Tarefas mais realizadas no período',fontsize = 20)
figura_tarefas.set_xlabel('Numero de atividades',fontsize=15)
figura_tarefas.set_ylabel('Tipos de Tarefas',fontsize = 15)
figura_tarefas

### Instalações com maior numero de atividades realizadas

Análise de atividades em ativos classificados como Instalações (infraestrutura).

In [ ]:
# Selecionar 'ativos' classificados como 'Instalações':
ativ_i = arquivo_final_periodo.loc[arquivo_final_periodo['Tipo de Ativo'].str.contains("Instalações",na = False)]

In [ ]:
# Atividades para 'tipos de ativo' classificados como 'instalações':
instalações_atividades_mais_realizadas = ativ_i[['Ativo','contador']].groupby('Ativo').sum()
instalações_atividades_mais_realizadas = instalações_atividades_mais_realizadas.sort_values(by=['contador'],ascending=False)
instalações_atividades_mais_realizadas.head(10)


In [ ]:
# Grafico Instalações com maior numero de atividades realizadas no período:

instalações_mais_realizadas = instalações_atividades_mais_realizadas.head(10)
indice_x = instalações_mais_realizadas ['contador'].head(10)
nomes_instalações = instalações_mais_realizadas.index
fig = plt.figure(figsize=(15,10))
sns.set_style('white')
figura_instalações = sns.barplot(x=indice_x,y=nomes_instalações ,palette = 'mako')
figura_instalações.set_title('As instalações de infraestrutura com o maior numero de atividades realizadas ',fontsize = 20)
figura_instalações.set_xlabel('Numero de atividades',fontsize=15)
figura_instalações.set_ylabel('Instalações',fontsize = 15)
figura_instalações 

### Instalações com menor numero de atividades realizadas

In [ ]:
# Atividades para 'tipos de ativo' classificados como 'instalações':
instalações_menos_realizadas = instalações_atividades_mais_realizadas.tail(10)
instalações_menos_realizadas

### Equipamentos com maior numero de atividades realizadas

Análise de atividades em ativos classificados como Equipamentos (máquinas).

In [ ]:
# Selecionar ativos classificados como 'Equipamentos':
ativ_e = arquivo_final_periodo.loc[arquivo_final_periodo['Tipo de Ativo'].str.contains("Equipamentos",na = False)]

In [ ]:
# Selecionar ativos classificados como equipamentos que possuem o maior numero de atividades realizadas cadastradas:
equipamentos_atividades_mais_realizadas = ativ_e[['Ativo','contador']].groupby('Ativo').sum()
equipamentos_atividades_mais_realizadas = equipamentos_atividades_mais_realizadas.sort_values(by=['contador'],ascending=False)
equipamentos_atividades_mais_realizadas.head(10)


In [ ]:
# Grafico Equipamentos com maior numero de atividades realizadas:

equipamentos_mais_realizados = equipamentos_atividades_mais_realizadas.head(10)
indice_x = equipamentos_mais_realizados ['contador']
nomes_equipamentos = equipamentos_mais_realizados.index
fig = plt.figure(figsize=(15,10))
sns.set_style('white')
figura_equipamentos = sns.barplot(x=indice_x,y=nomes_equipamentos ,palette = 'bone')
figura_equipamentos.set_title('Os 10 Equipamentos com maior numero de atividades realizadas',fontsize = 20)
figura_equipamentos.set_xlabel('Numero de atividades',fontsize=15)
figura_equipamentos.set_ylabel('Equipamentos',fontsize = 15)
figura_equipamentos 

### Equipamentos com menor numero de atividades realizadas

In [ ]:
equipamentos_menos_realizados = equipamentos_atividades_mais_realizadas.tail(30)
equipamentos_menos_realizados

---

##                                               ----------- Preventivas -------------

### Instalações com maior numero de atividades preventivas realizadas

In [ ]:
# Filtrar pelo tipo de tarefa classificada como preventiva:
prev_i = ativ_i.loc[ativ_i['Tipo de tarefa'].str.contains("Preventiva",na = False)]

In [ ]:
# Selecionar os 10 ativos com maior numero de atividades preventivas realizadas:
instalações_preventivas_mais_realizadas = prev_i[['Ativo','contador']].groupby('Ativo').sum()
instalações_preventivas_mais_realizadas = instalações_preventivas_mais_realizadas .sort_values(by=['contador'],ascending=False)
instalações_preventivas_mais_realizadas.head(10)

In [ ]:
# Selecionar os 10 ativos com menor numero de atividades preventivas realizadas:
instalações_menos_preventivas = instalações_preventivas_mais_realizadas.tail(10)
instalações_menos_preventivas

In [ ]:
# Mostrar informações das linhas para o ativo com a string 'Planta 1':
prev_i.loc[prev_i['Ativo'].str.contains("Planta 1",na = False)]

### Equipamentos com maior numero de atividades preventivas realizadas

In [ ]:
# Filtrar pelo 'tipo de tarefa' classificada como 'preventiva':
prev_e = ativ_e.loc[ativ_e['Tipo de tarefa'].str.contains("Preventiva",na = False)]

In [ ]:
# Selecionar os 10 ativos com maior numero de atividades preventivas realizadas:
ativos_preventivas_mais_realizadas = prev_e[['Ativo','contador']].groupby('Ativo').sum()
ativos_preventivas_mais_realizadas  = ativos_preventivas_mais_realizadas .sort_values(by=['contador'],ascending=False)
ativos_preventivas_mais_realizadas.head(10)


### Equipamentos com menor numero de atividades preventivas realizadas

In [ ]:
# Selecionar os 10 ativos com menor numero de atividades preventivas realizadas:
preventivas_menos_realizadas = ativos_preventivas_mais_realizadas.tail(30)
preventivas_menos_realizadas

### Atividades Preventivas X Mês X Ano 

In [ ]:
# Separação do numero de atividades preventivas por dia:
ativ_prev = arquivo_final_periodo.loc[arquivo_final_periodo['Tipo de tarefa'].str.contains("Preventiva",na = False)]
ativ_prev['Data final'] = pd.to_datetime(ativ_prev['Data final'], format= '%Y%m%d %H:%M:%S') 
                      
# Numero de atividades preventivas no ano analisado:
ativ_prev_periodo = ativ_prev[['Data final']]
ativ_prev_periodo = ativ_prev_periodo.groupby([ativ_prev_periodo['Data final'].dt.year.rename('Ano'), ativ_prev_periodo['Data final'].dt.month.rename('Mês')]).agg({'count'})
print (ativ_prev_periodo)
print("O numero de atividades preventivas totais foi de: ",ativ_prev_periodo['Data final'].sum())

---

##  ----------- Corretivas  -----------

### Instalações com o maior numero de Corretivas realizadas

In [ ]:
# Filtrar pelo tipo de tarefa classificada como corretiva
corret_i = ativ_i.loc[ativ_i['Tipo de tarefa'].str.contains("Corretiva",na = False)]

In [ ]:
# Selecionar os 10 ativos com maior numero de atividades corretivas realizadas:
instalações_corretivas_mais_realizadas = corret_i[['Ativo','contador']].groupby('Ativo').sum()
instalações_corretivas_mais_realizadas = instalações_corretivas_mais_realizadas .sort_values(by=['contador'],ascending=False)
instalações_corretivas_mais_realizadas.head(10)


In [ ]:
# Mostrar informações da linha para o equipamento "Planta 2"
corret_i.loc[corret_i['Ativo'].str.contains("Planta 2", na = False)]

In [ ]:
# Selecionar os 10 ativos com menor numero de atividades corretivas realizadas:
instalações_menos_corretivas = instalações_corretivas_mais_realizadas.tail(10)
instalações_menos_corretivas

### Equipamentos com maior numero de atividades corretivas realizadas

In [ ]:
# Selecionar os 10 equipametnos com maior numero de atividades corretivas realizadas:
corret_e = ativ_e.loc[ativ_e['Tipo de tarefa'].str.contains("Corretiva",na = False)]

equip_corretivas_mais_realizadas = corret_e[['Ativo','contador']].groupby('Ativo').sum()
equip_corretivas_mais_realizadas = equip_corretivas_mais_realizadas.sort_values(by=['contador'],ascending=False)
equip_corretivas_mais_realizadas.head(10)

In [ ]:
# Selecionar os 10 equipametnos com menor numero de atividades corretivas realizadas:
equip_menos_corretivas = equip_corretivas_mais_realizadas.tail(10)
equip_menos_corretivas

In [ ]:
#Criando um arquivo com o Tipo de Tarefa 'corretiva' para analise de palavras mais comuns, usada na parte de análise
# de falhas em equipamentos:
# Selecionando apenas 'Terefa'
corretiva_analise_tarefa = corret_e[['Tarefa']]
corretiva_analise_tarefa.to_csv('NOME DO ARQUIVO CSV .csv',index=False)
corretiva_analise_tarefa.head()

### Atividades Corretivas X Mês X Ano

In [ ]:
# Visualização das datas com maior numero de atividades corretivas no período:
ativ_corret = arquivo_final_periodo.loc[arquivo_final_periodo['Tipo de tarefa'].str.contains("Corretiva",na = False)]
ativ_corret['Data final'] = pd.to_datetime(ativ_corret['Data final'], format= '%Y%m%d %H:%M:%S')

# Numero de atividades corretivas no ano analisado:
ativ_corret_periodo = ativ_corret[['Data final']]
ativ_corret_periodo = ativ_corret_periodo.groupby([ativ_corret_periodo['Data final'].dt.year.rename('Ano'), ativ_corret_periodo['Data final'].dt.month.rename('Mês')]).agg({'count'})
print(ativ_corret_periodo)
print("O numero de atividades corretivas totais foi de: ",ativ_corret_periodo['Data final'].sum())


### Análise dos termos mais comuns para descrição das atividades realizadas

In [ ]:
# Habilitar findspark para uso do Spark
# Caminho para pacote instalado
findspark.init('CAMINHO EM QUE O PACOTE PYSPARK FOI SALVO')
# foi utilizado o' findspark.init' pois a função 'findspark.find()' não foi reconhecida.

In [ ]:
# Inicialização do contexto spark:
#sc1 = pyspark.SparkContext(appName='corretiva_tarefas_periodo')

In [ ]:
#Contagem de palavras com NLTK
nltk.download('stopwords')
nltk.download('punkt')

# Download do arquivo que será utilizado.
# O arquivo foi criado durante as análises de dados do período:
arquivo_analise_falha = 'file:CAMINHO EM QUE O ARQUIVO CSV COM AS TAREFAS FOI SALVO'
stopwords = nltk.corpus.stopwords.words('portuguese') # Uso do idioma portugues
punctuation = string.punctuation

In [ ]:
# Abrindo arquivo csv baixado:
with open("NOME DO ARQUIVO CSV COM AS TAREFAS", encoding="utf8") as text_file:
    contents = text_file.read()

In [ ]:
from collections import Counter

palavras = contents.lower()
palavras = palavras.replace('\n',' ').replace('\t','').split(' ')

contador = Counter(palavras)
    
for i in contador.items():
    print (i)

In [ ]:
import collections

# Lista de palavras
palavras = contents.lower()
palavras = palavras.replace('\n',' ').replace('\t','').split(' ')

# Contador para as ocorrencias de cada palavra
contador = Counter(palavras)

# As 100 palavras mais frequentes
contador = contador.most_common(100)

print (f'{contador}')

### Média de Falhas Atendidas Por Dia Durante o Período Por Equipamento

Como não possuímos os tempos corretos de operação de cada equipamento, aqui foi analisado o intervalo de dias entre falhas.

In [ ]:
#Listas
lista_maquina = []

# Codigo da função que permitirá digitar os equipamentos que se deseja analisar
def menu():
    #Variável:
    continuar = 1
    #Código condicional para permitir adicionar os nomes das máquinas que se deseja analisar:
    while(continuar):
        continuar = int(input("0. Sair\n"+ "1. Para inserir maquinas\n"))            
        if(continuar == 1):  # se o nuero digitado for 1, a condição é atendida.
            maquina = input("Digite as maquinas para calculo da média: ")
            lista_maquina.append(maquina)
    print(lista_maquina)

# Para ativar o codigo da função
menu() 

In [ ]:
# Listas
dias =[]
dias_maquina = []
media_t = []
lista_m = []

#Variáveis
m = 0
x = 0
i = 0

# Código condicional

for maq in lista_maquina:
    equip_corretiva_mfpd = arquivo_final_periodo.loc[arquivo_final_periodo['Tipo de tarefa'].str.contains("Corretiva",na = False)]
    equip_corretiva_mfpd = equip_corretiva_mfpd.loc[equip_corretiva_mfpd['Tipo de Ativo'].str.contains("Equipamento",na = False)]
    equip_corretiva_mfpd = equip_corretiva_mfpd.loc[~equip_corretiva_mfpd['Tipo de tarefa'].str.contains("molde|Molde",na = False)]
    equip_corretiva_mfpd = equip_corretiva_mfpd.loc[equip_corretiva_mfpd['Ativo'].str.contains(maq,na = False)]
    equip_corretiva_mfpd = equip_corretiva_mfpd.loc[~equip_corretiva_mfpd['Ativo'].str.contains("Planta 3|Planta 2",na = False)]
    equip_corretiva_mfpd['Data final'] = pd.to_datetime(equip_corretiva_mfpd['Data final'],infer_datetime_format=True) 

    equip_corretiva_mfpd_tempo = equip_corretiva_mfpd[['Tipo de Ativo','Data final']].groupby('Tipo de Ativo').sum()
    equip_corretiva_mfpd_tempo = equip_corretiva_mfpd.sort_values(by=['Data final'],ascending=False)


    m = equip_corretiva_mfpd_tempo['Data final'].count()
    lista_m.append(m)
    x = m-1
        
    for i in range(x):
        # Data final
        d2 = equip_corretiva_mfpd_tempo['Data final'].iloc[i]

        # Data inicial
        d1 = equip_corretiva_mfpd_tempo['Data final'].iloc[i+1]

        # Calculo da quantidade de dias
        quantidade_dias = abs((d2 - d1).days)
        dias.append(quantidade_dias)
        i = i+1
        
    dias_maquina.append(dias)
    soma_dias = sum(dias)
    media_dias = soma_dias/x
    media_t.append(media_dias)    
    dias =[] # lista dias é novamente zerada para guardar apenas as informações da próxima maquina analisada
    
# DataFrame com os dados resultante
df_mfpd=pd.DataFrame({"Maquina":lista_maquina,"Ocorrências":lista_m,"Média":media_t})


#Utilizamos a função abs para garantir que a quantidade de dias de diferença seja sempre positiva, 
#independente da ordem em que as datas foram subtraídas

In [ ]:
# Vizualização do DataFrame:
df_mfpd.sort_values(by=['Ocorrências'],ascending=False)

In [ ]:
# Visualização das atividades corretivas realizadas nas injetoras da 'Planta 1':
# As Injetoras foram os equipamentos com **maior** numero de ocorrências no período.

# !Obs : Foram escolhidas injetoras que permaneceram na planta durante os períodos estudados.

# Listas
lista_injetoras = ['INJETORA 01','INJETORA 10','INJETORA 11','INJETORA 12','INJETORA 13','INJETORA 18','INJETORA 24','INJETORA 30']
dias =[]
dias_maquina = []
media_t = []
lista_m = []

#Variáveis
m = 0
x = 0
i = 0

# Código condicional

for inj in lista_injetoras:
    equip_corretiva_mfpd = arquivo_final_periodo.loc[arquivo_final_periodo['Tipo de tarefa'].str.contains("Corretiva",na = False)]
    equip_corretiva_mfpd = equip_corretiva_mfpd.loc[equip_corretiva_mfpd['Tipo de Ativo'].str.contains("Equipamento",na = False)]
    equip_corretiva_mfpd = equip_corretiva_mfpd.loc[~equip_corretiva_mfpd['Tipo de tarefa'].str.contains("molde|Molde",na = False)]
    equip_corretiva_mfpd = equip_corretiva_mfpd.loc[equip_corretiva_mfpd['Ativo'].str.contains(inj,na = False)]
    equip_corretiva_mfpd = equip_corretiva_mfpd.loc[~equip_corretiva_mfpd['Ativo'].str.contains("Planta 3|Planta 2",na = False)]
    equip_corretiva_mfpd['Data final'] = pd.to_datetime(equip_corretiva_mfpd['Data final'],infer_datetime_format=True) 

    equip_corretiva_mfpd_tempo = equip_corretiva_mfpd[['Tipo de Ativo','Data final']].groupby('Tipo de Ativo').sum()
    equip_corretiva_mfpd_tempo = equip_corretiva_mfpd.sort_values(by=['Data final'],ascending=False)


    m = equip_corretiva_mfpd_tempo['Data final'].count()
    lista_m.append(m)
    x = m-1
        
    for i in range(x):
        # Data final
        d2 = equip_corretiva_mfpd_tempo['Data final'].iloc[i]

        # Data inicial
        d1 = equip_corretiva_mfpd_tempo['Data final'].iloc[i+1]

        # Calculo da quantidade de dias
        quantidade_dias = abs((d2 - d1).days)
        dias.append(quantidade_dias)
        i = i+1
        
    dias_maquina.append(dias)
    soma_dias = sum(dias)
    media_dias = soma_dias/x
    media_t.append(media_dias)    
    dias =[] # lista dias é novamente zerada para guardar apenas as informações da próxima maquina analisada
    
# DataFrame com os dados resultante
df_inj=pd.DataFrame({"Maquina":lista_injetoras,"Ocorrências":lista_m,"Média":media_t})

# Vizualização do DataFrame:
df_inj = df_inj.sort_values(by=['Ocorrências'],ascending=False)
df_inj

In [ ]:
# Visualização das atividades corretivas realizadas nas injetora 01 da 'Planta 1':
# A Injetora 01 foi o equipamento com **maior** numero de ocorrências no período.
equip_corretiva_mfpd = arquivo_final_periodo.loc[arquivo_final_periodo['Tipo de tarefa'].str.contains("Corretiva",na = False)]
equip_corretiva_mfpd = equip_corretiva_mfpd.loc[equip_corretiva_mfpd['Tipo de Ativo'].str.contains("Equipamento",na = False)]
equip_corretiva_mfpd = equip_corretiva_mfpd.loc[~equip_corretiva_mfpd['Tipo de tarefa'].str.contains("molde|Molde",na = False)]
equip_corretiva_mfpd = equip_corretiva_mfpd.loc[equip_corretiva_mfpd['Ativo'].str.contains('INJETORA 01',na = False)]
equip_corretiva_mfpd = equip_corretiva_mfpd.loc[~equip_corretiva_mfpd['Ativo'].str.contains("Planta 3|Planta 2",na = False)]
equip_corretiva_mfpd['Data final'] = pd.to_datetime(equip_corretiva_mfpd['Data final'],infer_datetime_format=True) 

equip_corretiva_mfpd_tempo = equip_corretiva_mfpd[['Tipo de Ativo','Data final']].groupby('Tipo de Ativo').sum()
equip_corretiva_mfpd_tempo = equip_corretiva_mfpd.sort_values(by=['Data final'],ascending=False)

equip_corretiva_mfpd_tempo.head(10)

---

##  ----------- Preventivas x Corretivas  -----------

In [ ]:
# Criação de DataFrame com os numeros de Preventivas e Corretivas por mês no período:
m = pd.merge(ativ_prev_periodo, ativ_corret_periodo , how = 'inner', on = 'Mês')
m.columns = ['Preventivas','Corretivas']
m

In [ ]:
# Grafico com os numeros de atividades Preventivas e Corretivas por mês no período:
prev_b = m['Preventivas']
corret_b = m['Corretivas']
resultado = pd.crosstab(prev_b,corret_b)
indice = m.index
fig, ax = plt.subplots(figsize = [15,10])
x = np.arange(len(indice))
ax.set_xticks(x)
ax.set_xticklabels(indice)
ax.bar(x-0.2,prev_b,width = 0.4)
ax.bar(x+0.2,corret_b,width = 0.4)
# width é para alterar a largura da minha barra
# deslocando o x nós conseguimos colocar uma barra do lado da outra
plt.legend(['Preventivas','Corretivas'])

In [ ]:
# Inserção de colunas com a si=oma de atividades e com a porcentages de cada atividade por mês:
m.insert(loc=2,column='soma',value= m.sum(axis=1))
por_prev = m['Preventivas']/m['soma'] * 100
por_corret = m['Corretivas']/m['soma'] *100
m.insert(loc=3,column='% Prev',value= por_prev)
m.insert(loc=4,column='% Corret',value= por_corret)
m

In [ ]:
# Grafico %Preventivas x %Corretivas por mês no período:
y = m.index
prev = m['% Prev']
corret= m['% Corret']
plt.figure(figsize=(15, 10))
plt.plot(y,prev,label = '% Preventiva',linewidth=2)
plt.plot(y,corret,label = '% Corretiva',linewidth=2)
plt.legend(fontsize=10)

plt.title('Preventiva x Corretiva',fontsize=20)
plt.xlabel('Mês',fontsize=15)
plt.ylabel('Porcentagem',fontsize=15)


---

##  ----------- Melhorias  -----------

### Instalações com o maior numero de melhorias realizadas

In [ ]:
# Filtrar pelo tipo de tarefa classificada como melhoria em instalações
mel_i = ativ_i.loc[ativ_i['Tipo de tarefa'].str.contains("melhoria|Melhoria",na = False)]

In [ ]:
# Selecionar os 10 ativos com maior numero de atividades melhorias realizadas:
instalações_melhorias_mais_realizadas = mel_i[['Ativo','contador']].groupby('Ativo').sum()
instalações_melhorias_mais_realizadas = instalações_melhorias_mais_realizadas .sort_values(by=['contador'],ascending=False)
instalações_melhorias_mais_realizadas.head(10)


In [ ]:
# Selecionar os 10 ativos com menor numero de atividades melhorias realizadas:
instalações_menos_melhorias = instalações_melhorias_mais_realizadas.tail(10)
instalações_menos_melhorias

### Equipamentos com maior numero de melhorias realizadas

In [ ]:
# Selecionar os equipamentos com maior numero de atividades classificadas como melhoria:
mel_e = ativ_e.loc[ativ_e['Tipo de tarefa'].str.contains("melhoria|Melhoria",na = False)]

equip_melhorias_mais_realizadas = mel_e[['Ativo','contador']].groupby('Ativo').sum()
equip_melhorias_mais_realizadas = equip_melhorias_mais_realizadas.sort_values(by=['contador'],ascending=False)
equip_melhorias_mais_realizadas.head(10)

In [ ]:
# Mostrar informações da linha para o equipamento "linha 716"
mel_e.loc[mel_e['Ativo'].str.contains("LINHA 716", na = False)]

In [ ]:
# Selecionar os 10 equipametnos com menor numero de melhorias realizadas:
equip_menos_melhorias = equip_melhorias_mais_realizadas.tail(10)
equip_menos_melhorias

---